In [68]:
from app.log import logging_config
from os.path import dirname, join
from dotenv import load_dotenv
import logging
from app.util.message import starting_message
from app import yahoo_finance, Polygon, Alphavantage
import pandas as pd
import time
from datetime import datetime

In [2]:
#print(join(dirname(__file__), '.env'))
#load_dotenv(dotenv_path=join(dirname(__file__), '.env'))
load_dotenv()
from os import environ
print(environ.get("POLYGON_KEY"))

PWRjSfmBc9xYnPZMQhA9ukDOuOIId90D


In [3]:
logging_config.init_logging()
__LOG = logging.getLogger(__name__)
__LOG.info('...... Initialization Completed  ......')
__LOG.info(starting_message())

# Yahoo Data

In [110]:
status, yahoo_data = yahoo_finance.market_value('BTC-USD')
if status:
    print(yahoo_data)

[*********************100%***********************]  1 of 1 completed
                    Open          High           Low         Close  \
Date                                                                 
2019-12-09   7268.902832   7308.836426   6924.375977   7152.301758   
2019-12-16   7153.663086   7518.033203   6540.049316   7511.588867   
2019-12-23   7508.902344   7656.176270   7189.934082   7422.652832   
2019-12-30   7420.272949   7544.497070   6914.996094   7411.317383   
2020-01-06   7410.452148   8396.738281   7409.292969   8192.494141   
...                  ...           ...           ...           ...   
2022-05-09  34060.015625  34222.074219  26350.490234  31305.113281   
2022-05-16  31304.375000  31305.341797  28708.955078  30323.722656   
2022-05-23  30309.396484  30590.585938  28261.906250  29445.957031   
2022-05-30  29443.365234  32249.863281  29303.572266  31792.310547   
2022-06-02  29753.320312  29843.408203  29664.839844  29803.189453   

               Adj C

In [20]:
yahoodf = pd.DataFrame(yahoo_data, columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
#yahoodf['Date'] = yahoodf.index
yahoodf = yahoodf.reset_index(level=0)
yahoodf.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-12-09,7268.902832,7308.836426,6924.375977,7152.301758,7152.301758,34018159534
1,2019-12-16,7153.663086,7518.033203,6540.049316,7511.588867,7511.588867,165399102665
2,2019-12-23,7508.902344,7656.176270,7189.934082,7422.652832,7422.652832,161758217053
3,2019-12-30,7420.272949,7544.497070,6914.996094,7411.317383,7411.317383,149690652648
4,2020-01-06,7410.452148,8396.738281,7409.292969,8192.494141,8192.494141,184901289966


## aca va la explicacion de cada columna de yahoo

In [28]:
yahoodf.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [29]:
yahoodf.describe()

,Open,High,Low,Close,Adj Close,Volume
count,131.000000,131.000000,131.000000,131.000000,131.000000,1.310000e+02
mean,30261.964534,32545.497454,27885.478952,30452.241617,30452.241617,2.603603e+11
std,18623.231915,19842.716618,16952.983957,18514.649756,18514.649756,1.136552e+11
min,5385.229492,6844.261719,4106.980957,5392.314941,5392.314941,3.401816e+10
25%,10026.060059,10518.232422,9813.178223,10198.512695,10198.512695,1.879022e+11
50%,34254.015625,36542.109375,29664.839844,34240.187500,34240.187500,2.391808e+11
75%,46430.585938,48445.232422,42434.947266,46454.812500,46454.812500,3.011762e+11
max,65521.289062,68789.625000,62333.914062,65466.839844,65466.839844,7.666798e+11


In [41]:
countAnalisis = yahoodf[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]].count()
countAnalisis

Date         131
Open         131
High         131
Low          131
Close        131
Adj Close    131
Volume       131
dtype: int64

In [43]:
meanAnalisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].mean()
meanAnalisis

Open         3.026196e+04
High         3.254550e+04
Low          2.788548e+04
Close        3.045224e+04
Adj Close    3.045224e+04
Volume       2.603603e+11
dtype: float64

In [44]:
minAnalisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].min()
minAnalisis

Open         5.385229e+03
High         6.844262e+03
Low          4.106981e+03
Close        5.392315e+03
Adj Close    5.392315e+03
Volume       3.401816e+10
dtype: float64

In [45]:
maxAnalisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].max()
maxAnalisis

Open         6.552129e+04
High         6.878962e+04
Low          6.233391e+04
Close        6.546684e+04
Adj Close    6.546684e+04
Volume       7.666798e+11
dtype: float64

In [46]:
q25Analisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].quantile(0.25)
q25Analisis

Open         1.002606e+04
High         1.051823e+04
Low          9.813178e+03
Close        1.019851e+04
Adj Close    1.019851e+04
Volume       1.879022e+11
Name: 0.25, dtype: float64

In [47]:
q50Analisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].quantile(0.5)
q50Analisis

Open         3.425402e+04
High         3.654211e+04
Low          2.966484e+04
Close        3.424019e+04
Adj Close    3.424019e+04
Volume       2.391808e+11
Name: 0.5, dtype: float64

In [48]:
q75Analisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].quantile(0.75)
q75Analisis

Open         4.643059e+04
High         4.844523e+04
Low          4.243495e+04
Close        4.645481e+04
Adj Close    4.645481e+04
Volume       3.011762e+11
Name: 0.75, dtype: float64

In [100]:
q95nalisis = yahoodf[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].quantile(0.95)
q95nalisis

Open         5.901415e+04
High         6.222050e+04
Low          5.475516e+04
Close        5.903044e+04
Adj Close    5.903044e+04
Volume       4.895026e+11
Name: 0.95, dtype: float64

## Falta generar Graficas interesantes EDA

# Falta Analisis a cada paso


# Polygon

In [179]:
polygon=Polygon(environ.get("POLYGON_KEY"))
#print('polygon')
status, polygon_data = polygon.get_market_status()
if status:
    #print(polygon_data)
    polygon_market_status = pd.DataFrame.from_dict(polygon_data)
polygon_market_status = polygon_market_status.reset_index(level=0)
polygon_market_status

,index,market,earlyHours,afterHours,serverTime,exchanges,currencies
0,nyse,closed,False,False,2022-06-02T21:03:21-04:00,closed,NaN
1,nasdaq,closed,False,False,2022-06-02T21:03:21-04:00,closed,NaN
2,otc,closed,False,False,2022-06-02T21:03:21-04:00,closed,NaN
3,fx,closed,False,False,2022-06-02T21:03:21-04:00,NaN,open
4,crypto,closed,False,False,2022-06-02T21:03:21-04:00,NaN,open


In [180]:
polygon_market_status['serverTime'] = pd.to_datetime(polygon_market_status['serverTime'])
polygon_market_status.dtypes

index                                         object
market                                        object
earlyHours                                      bool
afterHours                                      bool
serverTime    datetime64[ns, pytz.FixedOffset(-240)]
exchanges                                     object
currencies                                    object
dtype: object

In [181]:
polygon_market_status.describe()

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\3545403409.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  polygon_market_status.describe()


,index,market,earlyHours,afterHours,serverTime,exchanges,currencies
count,5,5,5,5,5,3,2
unique,5,1,1,1,1,1,1
top,nyse,closed,False,False,2022-06-02 21:03:21-04:00,closed,open
freq,1,5,5,5,5,3,2
first,NaN,NaN,NaN,NaN,2022-06-02 21:03:21-04:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2022-06-02 21:03:21-04:00,NaN,NaN


In [63]:
# Esta no nos dio informacion relevante
status, polygon_data = polygon.get_news_ticker('2022-06-02', 'BTC')
if status:
    print(polygon_data)
    polygon_news_ticker = pd.DataFrame.from_dict(polygon_data)
polygon_news_ticker

{'results': [], 'status': 'OK', 'request_id': '8f70ab1902e481cc31d235a4a8b2b52e'}


,results,status,request_id


In [64]:
status, polygon_data_on = polygon.get_old_news('2022-05-31')
#print(status)
if status:
    #print(polygon_data)
    polygon_old_news = pd.DataFrame.from_dict(polygon_data_on["results"])
#polygon_old_news
polygon_old_news = polygon_old_news[['title', 'author', 'published_utc', 'tickers', 'description', 'keywords']]
n_columns_news = {'title':'Title', 'author':'Author', 'published_utc':'Date', 'tickers':'Tickers', 'description':'Description', 'keywords':'Keywords'}
polygon_old_news.rename(columns = n_columns_news, inplace = True)
polygon_old_news.head()

,Title,Author,Date,Tickers,Description,Keywords
0,The Limestone Boat Company Announces First Qua...,The Limestone Boat Company Limited,2022-05-31T01:51:00Z,[BOAT],"Limestone continues to grow unit production, r...",[Earnings Releases and Operating Results]
1,Shiba Inu (SHIB) Founder Ryoshi Abruptly Delet...,Samyuktha Sriram,2022-05-31T02:52:56Z,[TWTR],"Ryoshi, the pseudonymous creator behind meme c...","[News, Cryptocurrency, Social Media, Top Stori..."
2,Apple Will Have To Face Antitrust Lawsuit From...,Shivdeep Dhaliwal,2022-05-31T03:16:09Z,[AAPL],Apple Inc’s (NASDAQ: AAPL) attempts to have an...,"[News, Legal, Tech, Media]"
3,"Why Alibaba, Nio, Chinese Peers Are Struggling...",Navdeep Yadav,2022-05-31T04:07:22Z,"[BABA, JD, NIO, XPEV]",Shares of U.S.-listed Chinese companies were n...,"[News, Asia, Top Stories, Markets, Movers, Tec..."
4,Coinbase Institutional Clients Raised Bitcoin ...,Samyuktha Sriram,2022-05-31T04:20:15Z,[COIN],Data shows that institutions on Coinbase Globa...,"[News, Cryptocurrency, Markets]"


In [176]:
polygon_old_news['Date'] = pd.to_datetime(polygon_old_news['Date'])
polygon_old_news.dtypes

Title                       object
Author                      object
Date           datetime64[ns, UTC]
Tickers                     object
Description                 object
Keywords                    object
dtype: object

In [177]:
polygon_old_news.describe()

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\2004128356.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  polygon_old_news.describe()


,Title,Author,Date,Tickers,Description,Keywords
count,1000,1000,1000,1000,980,563
unique,992,369,644,861,950,147
top,Why This 1 Growth Stock Could Be a Great Addit...,Zacks Equity Research,2022-05-31 20:05:00+00:00,[TSLA],"Whether you're a value, growth, or momentum in...",[investing]
freq,3,311,19,9,11,119
first,NaN,NaN,2022-05-31 01:51:00+00:00,NaN,NaN,NaN
last,NaN,NaN,2022-06-01 13:40:08+00:00,NaN,NaN,NaN


In [178]:
countAnalisis_polygon_old_news = polygon_old_news[["Title", "Author", "Date", "Tickers", "Description", "Keywords"]].count()
countAnalisis_polygon_old_news

Title          1000
Author         1000
Date           1000
Tickers        1000
Description     980
Keywords        563
dtype: int64

In [65]:
status, polygon_data_on = polygon.get_news('2022-05-31')
#print(status)
if status:
    #print(polygon_data)
    polygon_get_news = pd.DataFrame.from_dict(polygon_data_on["results"])
#polygon_get_news.head()
polygon_get_news = polypolygon_old_news.describe()
polygon_old_news.dtypesgon_get_news[['title', 'author', 'published_utc', 'tickers', 'description', 'keywords']]
polygon_get_news.rename(columns = n_columns_news, inplace = True)
polygon_get_news.head()

,Title,Author,Date,Tickers,Description,Keywords
0,California Water Service Receives CPUC Approva...,California Water Service Group,2022-06-02T23:15:00Z,[CWT],"SAN JOSE, Calif., June 02, 2022 (GLOBE NEWSW...",[Mergers and Acquisitions]
1,Aduro Clean Technologies fera des présentation...,Aduro Clean Technologies Inc.,2022-06-02T22:52:00Z,[ACT],Aduro fera une présentation lors du Global Ene...,[Calendar of Events]
2,Aduro Clean Technologies präsentiert auf anste...,Aduro Clean Technologies Inc.,2022-06-02T22:52:00Z,[ACT],Aduro stellt Technologie auf der Global Energy...,[Calendar of Events]
3,Why I'm not celebrating National Doughnut Day,MarketWatch,2022-06-02T22:46:00Z,[DNUT],It has become increasingly challenging to find...,NaN
4,The Toro Company (TTC) Q2 2022 Earnings Call T...,newsfeedback@fool.com (Motley Fool Transcribing),2022-06-02T22:30:42Z,[TTC],TTC earnings call for the period ending March ...,[earningscall-transcripts]


In [173]:
polygon_get_news['Date'] = pd.to_datetime(polygon_get_news['Date'])
polygon_get_news.dtypes

Title                       object
Author                      object
Date           datetime64[ns, UTC]
Tickers                     object
Description                 object
Keywords                    object
dtype: object

In [174]:
polygon_get_news.describe()

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\740367754.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  polygon_get_news.describe()


,Title,Author,Date,Tickers,Description,Keywords
count,1000,1000,1000,1000,984,495
unique,997,311,608,856,963,133
top,Breaking Down the Tech Sector's Earnings Outlook,Zacks Equity Research,2022-06-02 20:05:00+00:00,[DJIA],"Here at Zacks, our focus is on the proven Zack...",[investing]
freq,2,379,20,8,9,134
first,NaN,NaN,2022-06-01 20:30:00+00:00,NaN,NaN,NaN
last,NaN,NaN,2022-06-02 23:15:00+00:00,NaN,NaN,NaN


In [175]:
countAnalisis_polygon_get_news = polygon_get_news[["Title", "Author", "Date", "Tickers", "Description", "Keywords"]].count()
countAnalisis_polygon_get_news

Title          1000
Author         1000
Date           1000
Tickers        1000
Description     984
Keywords        495
dtype: int64

In [66]:
daily_results = pd.DataFrame()
#dates3 = ['2022-01-07', '2022-01-14', '2022-01-21', '2022-01-28', '2022-02-04', '2022-02-11', '2022-02-18', '2022-02-25', '2022-03-04', '2022-03-11', '2022-03-18', '2022-03-25', '2022-04-01', '2022-04-08', '2022-04-15', '2022-04-22', '2022-04-29', '2022-05-06', '2022-05-13', '2022-05-20', '2022-05-27']
dates = ['2022-01-07', '2022-01-14', '2022-01-21', '2022-01-28', '2022-02-04', '2022-02-11', '2022-02-18', '2022-02-25', '2022-03-04', '2022-03-11', '2022-03-18', '2022-03-25', '2022-04-01', '2022-04-08', '2022-04-15', '2022-04-22', '2022-04-29', '2022-05-06', '2022-05-13', '2022-05-20', '2022-05-27']

In [67]:
for element in dates:
    #print(element)
    status, polygon_data = polygon.daily_result('BTC', element)
    if status:
        #print(polygon_data)
        polygon_daily_result = pd.DataFrame.from_dict(polygon_data)
    polygon_daily_result = polygon_daily_result.head(1)
    #polygon_daily_result

    polygon_daily_result_open = pd.DataFrame.from_dict(polygon_daily_result['openTrades'][0])

    n_columns_news3 = {'x':'ExchangeOpen', 'p':'PriceOpen', 's':'VolumeOpen', 'c':'ConditionCodesOpen', 'i':'IdOpen', 't':'TimestampOpen'}
    polygon_daily_result_open.rename(columns = n_columns_news3, inplace = True)
    #polygon_daily_result_open.head()

    polygon_daily_result_close = pd.DataFrame.from_dict(polygon_daily_result['closingTrades'][0])


    n_columns_news2 = {'x':'ExchangeClose', 'p':'PriceClose', 's':'VolumeClose', 'c':'ConditionCodesClose', 'i':'IdClose', 't':'TimestampClose'}
    polygon_daily_result_close.rename(columns = n_columns_news2, inplace = True)
    #polygon_daily_result_close.head()

    polygon_daily_result = polygon_daily_result[['symbol', 'isUTC', 'day', 'open', 'close']]
    #polygon_daily_result.join(polygon_daily_result_open)
    polygon_daily_result = pd.concat([polygon_daily_result,polygon_daily_result_open], axis = 1)
    polygon_daily_result = pd.concat([polygon_daily_result,polygon_daily_result_close], axis = 1)
    #polygon_daily_result.head()

    daily_results = daily_results.append(polygon_daily_result, ignore_index=True)
    time.sleep(15)
daily_results.head()

,symbol,isUTC,day,open,close,ExchangeOpen,PriceOpen,VolumeOpen,ConditionCodesOpen,IdOpen,TimestampOpen,ExchangeClose,PriceClose,VolumeClose,ConditionCodesClose,IdClose,TimestampClose
0,BTC-USD,True,2022-01-07T00:00:00Z,43081.79,41564.61,1,43081.79,0.000675,2,260333638,1641513600010,1,41564.61,0.000247,1,260893471,1641599999913
1,BTC-USD,True,2022-01-14T00:00:00Z,42576.27,43086.89,1,42576.27,0.001000,2,263672878,1642118400058,1,43086.89,0.000779,1,264133049,1642204799588
2,BTC-USD,True,2022-01-21T00:00:00Z,40671.77,36456.96,1,40671.77,0.000120,2,266526967,1642723200089,1,36456.96,0.001000,2,267665914,1642809599827
3,BTC-USD,True,2022-01-28T00:00:00Z,37188.37,37741.59,1,37188.37,0.000080,1,271657003,1643328000218,1,37741.59,0.000960,1,272374175,1643414399859
4,BTC-USD,True,2022-02-04T00:00:00Z,37367.00,41614.66,2,37367.00,0.005000,2,991190639,1643932800019,1,41614.66,0.006329,2,276280906,1644019199952


In [98]:
daily_results['DateOpen'] = pd.to_datetime(daily_results['TimestampOpen']/1000, unit='s')
daily_results['DateClose'] = pd.to_datetime(daily_results['TimestampClose']/1000, unit='s')
daily_results.head()

,symbol,isUTC,day,open,close,ExchangeOpen,PriceOpen,VolumeOpen,ConditionCodesOpen,IdOpen,TimestampOpen,ExchangeClose,PriceClose,VolumeClose,ConditionCodesClose,IdClose,TimestampClose,DateOpen,DateClose
0,BTC-USD,True,2022-01-07T00:00:00Z,43081.79,41564.61,1,43081.79,0.000675,2,260333638,1641513600010,1,41564.61,0.000247,1,260893471,1641599999913,2022-01-07 00:00:00.009999872,2022-01-07 23:59:59.913000192
1,BTC-USD,True,2022-01-14T00:00:00Z,42576.27,43086.89,1,42576.27,0.001000,2,263672878,1642118400058,1,43086.89,0.000779,1,264133049,1642204799588,2022-01-14 00:00:00.058000128,2022-01-14 23:59:59.588000000
2,BTC-USD,True,2022-01-21T00:00:00Z,40671.77,36456.96,1,40671.77,0.000120,2,266526967,1642723200089,1,36456.96,0.001000,2,267665914,1642809599827,2022-01-21 00:00:00.088999936,2022-01-21 23:59:59.826999808
3,BTC-USD,True,2022-01-28T00:00:00Z,37188.37,37741.59,1,37188.37,0.000080,1,271657003,1643328000218,1,37741.59,0.000960,1,272374175,1643414399859,2022-01-28 00:00:00.217999872,2022-01-28 23:59:59.859000064
4,BTC-USD,True,2022-02-04T00:00:00Z,37367.00,41614.66,2,37367.00,0.005000,2,991190639,1643932800019,1,41614.66,0.006329,2,276280906,1644019199952,2022-02-04 00:00:00.019000064,2022-02-04 23:59:59.952000000


In [161]:
daily_results['day'] = pd.to_datetime(daily_results['day'])
daily_results.dtypes

symbol                              object
isUTC                                 bool
day                    datetime64[ns, UTC]
open                               float64
close                              float64
ExchangeOpen                         int64
PriceOpen                          float64
VolumeOpen                         float64
ConditionCodesOpen                   int64
IdOpen                              object
TimestampOpen                        int64
ExchangeClose                        int64
PriceClose                         float64
VolumeClose                        float64
ConditionCodesClose                  int64
IdClose                             object
TimestampClose                       int64
DateOpen                    datetime64[ns]
DateClose                   datetime64[ns]
dtype: object

In [162]:
daily_results.describe()

,open,close,ExchangeOpen,PriceOpen,VolumeOpen,ConditionCodesOpen,TimestampOpen,ExchangeClose,PriceClose,VolumeClose,ConditionCodesClose,TimestampClose
count,21.000000,21.000000,21.00000,21.000000,21.000000,21.000000,2.100000e+01,21.000000,21.000000,21.000000,21.000000,2.100000e+01
mean,39252.024525,38882.240485,2.52381,39252.024525,0.035497,1.476190,1.647562e+12,1.047619,38882.240485,0.002375,1.523810,1.647648e+12
std,4729.788359,4807.107933,4.82306,4729.788359,0.135182,0.601585,3.752685e+09,0.218218,4807.107933,0.003583,0.511766,3.752685e+09
min,28941.950000,28598.310000,1.00000,28941.950000,0.000080,0.000000,1.641514e+12,1.000000,28598.310000,0.000009,1.000000,1.641600e+12
25%,37367.000000,37741.590000,1.00000,37367.000000,0.000294,1.000000,1.644538e+12,1.000000,37741.590000,0.000436,1.000000,1.644624e+12
50%,40483.380000,39712.250000,1.00000,40483.380000,0.000710,2.000000,1.647562e+12,1.000000,39712.250000,0.001000,2.000000,1.647648e+12
75%,42576.270000,41774.980000,2.00000,42576.270000,0.004700,2.000000,1.650586e+12,1.000000,41774.980000,0.002710,2.000000,1.650672e+12
max,45525.250000,46296.340000,23.00000,45525.250000,0.621366,2.000000,1.653610e+12,2.000000,46296.340000,0.014692,2.000000,1.653696e+12


In [163]:
countAnalisis_daily_results = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].count()
countAnalisis_daily_results 

symbol                 21
isUTC                  21
day                    21
open                   21
close                  21
ExchangeOpen           21
PriceOpen              21
VolumeOpen             21
ConditionCodesOpen     21
IdOpen                 20
TimestampOpen          21
ExchangeClose          21
PriceClose             21
VolumeClose            21
ConditionCodesClose    21
IdClose                21
TimestampClose         21
dtype: int64

In [164]:
meanAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].mean()
meanAnalisis_daily_results 

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\1323079814.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  meanAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].mean()
C:\Users\julia\AppData\Local\Temp\ipykernel_13564\1323079814.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  meanAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "Volume

isUTC                   1.000000e+00
open                    3.925202e+04
close                   3.888224e+04
ExchangeOpen            2.523810e+00
PriceOpen               3.925202e+04
VolumeOpen              3.549680e-02
ConditionCodesOpen      1.476190e+00
TimestampOpen           1.647562e+12
ExchangeClose           1.047619e+00
PriceClose              3.888224e+04
VolumeClose             2.374596e-03
ConditionCodesClose     1.523810e+00
IdClose                1.242350e+188
TimestampClose          1.647648e+12
dtype: float64

In [166]:
minAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].min()
minAnalisis_daily_results  

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\2371734419.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  minAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].min()


symbol                                   BTC-USD
isUTC                                       True
day                    2022-01-07 00:00:00+00:00
open                                    28941.95
close                                   28598.31
ExchangeOpen                                   1
PriceOpen                               28941.95
VolumeOpen                               0.00008
ConditionCodesOpen                             0
TimestampOpen                      1641513600010
ExchangeClose                                  1
PriceClose                              28598.31
VolumeClose                             0.000009
ConditionCodesClose                            1
IdClose                               1109149659
TimestampClose                     1641599999913
dtype: object

In [167]:
maxAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].max()
maxAnalisis_daily_results 

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\1164570207.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  maxAnalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].max()


symbol                                   BTC-USD
isUTC                                       True
day                    2022-05-27 00:00:00+00:00
open                                    45525.25
close                                   46296.34
ExchangeOpen                                  23
PriceOpen                               45525.25
VolumeOpen                              0.621366
ConditionCodesOpen                             2
TimestampOpen                      1653609600059
ExchangeClose                                  2
PriceClose                              46296.34
VolumeClose                             0.014692
ConditionCodesClose                            2
IdClose                                345110241
TimestampClose                     1653695999947
dtype: object

In [169]:
#q25Analisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].quantile(0.25)
#q25Analisis_daily_results 

In [170]:
#q50Analisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].quantile(0.5)
#q50Analisis_daily_results 

In [171]:
#q75Analisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].quantile(0.75)
#q75Analisis_daily_results 

In [172]:
#q95nalisis_daily_results  = daily_results[["symbol", "isUTC", "day", "open", "close", "ExchangeOpen",	"PriceOpen", "VolumeOpen", "ConditionCodesOpen", "IdOpen", "TimestampOpen", "ExchangeClose", "PriceClose", "VolumeClose", "ConditionCodesClose", "IdClose", "TimestampClose"]].quantile(0.95)
#q95nalisis_daily_results

In [75]:
status, polygon_data = polygon.previous_close()
if status:
    #print(polygon_data)
    polygon_previous_close = pd.DataFrame.from_dict(polygon_data['results'])
n_columns_news4 = {'T':'ExchangeSymbol', 'v':'TradingVolume', 'vw':'AvgPriceVolume', 'o':'OpenPrice', 'c':'ClosePrice', 'h':'HighestPrice', 'l':'LowestPrice', 't':'TimeStamp', 'n':'#TransactionAggregateWindow'}
polygon_previous_close.rename(columns = n_columns_news4, inplace = True)
polygon_previous_close['Date'] = pd.to_datetime(polygon_previous_close['TimeStamp']/1000, unit='s')
#polygon_previous_close['Date'] = datetime.fromtimestamp(polygon_previous_close['TimeStamp'])
polygon_previous_close

,ExchangeSymbol,TradingVolume,AvgPriceVolume,OpenPrice,ClosePrice,HighestPrice,LowestPrice,TimeStamp,#TransactionAggregateWindow,Date
0,X:BTCUSD,36869.61995,30692.1045,31784.18,29833,31974.161427,29308.01,1654127999999,763425,2022-06-01 23:59:59.999000064


In [89]:
polygon_previous_close.dtypes

ExchangeSymbol                         object
TradingVolume                         float64
AvgPriceVolume                        float64
OpenPrice                             float64
ClosePrice                              int64
HighestPrice                          float64
LowestPrice                           float64
TimeStamp                               int64
#TransactionAggregateWindow             int64
Date                           datetime64[ns]
dtype: object

In [88]:
polygon_previous_close.describe()

,TradingVolume,AvgPriceVolume,OpenPrice,ClosePrice,HighestPrice,LowestPrice,TimeStamp,#TransactionAggregateWindow
count,1.00000,1.0000,1.00,1.0,1.000000,1.00,1.000000e+00,1.0
mean,36869.61995,30692.1045,31784.18,29833.0,31974.161427,29308.01,1.654128e+12,763425.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,36869.61995,30692.1045,31784.18,29833.0,31974.161427,29308.01,1.654128e+12,763425.0
25%,36869.61995,30692.1045,31784.18,29833.0,31974.161427,29308.01,1.654128e+12,763425.0
50%,36869.61995,30692.1045,31784.18,29833.0,31974.161427,29308.01,1.654128e+12,763425.0
75%,36869.61995,30692.1045,31784.18,29833.0,31974.161427,29308.01,1.654128e+12,763425.0
max,36869.61995,30692.1045,31784.18,29833.0,31974.161427,29308.01,1.654128e+12,763425.0


# AlphaAvantage

In [129]:
alphavantage=Alphavantage(environ.get("ALPHA_KEY"))
#print('polygon')
status, alphavantage_data = alphavantage.intraday('ETH')
if status:
    #print(alphavantage_data)
    alphavantage_intradayETH = pd.DataFrame.from_dict(alphavantage_data['Time Series Crypto (5min)'])
alphavantage_intradayETH = alphavantage_intradayETH.T
alphavantage_intradayETH = alphavantage_intradayETH.reset_index(level=0)
n_columns_rename = {'index':'DateTime','1. open':'Open', '2. high':'High', '3. low':'Low', '4. close':'Close', '5. volume':'Volume'}
alphavantage_intradayETH.rename(columns = n_columns_rename, inplace = True)
alphavantage_intradayETH

,DateTime,Open,High,Low,Close,Volume
0,2022-06-03 00:45:00,1824.80000,1825.93000,1822.30000,1825.19000,1257
1,2022-06-03 00:40:00,1827.47000,1828.32000,1824.64000,1824.81000,1392
2,2022-06-03 00:35:00,1832.64000,1833.46000,1826.33000,1827.47000,1683
3,2022-06-03 00:30:00,1833.52000,1835.79000,1832.54000,1832.63000,1407
4,2022-06-03 00:25:00,1832.71000,1833.75000,1829.38000,1833.52000,1715
...,...,...,...,...,...,...
95,2022-06-02 16:50:00,1820.81000,1831.66000,1819.76000,1830.40000,3314
96,2022-06-02 16:45:00,1826.50000,1828.50000,1819.73000,1820.82000,3409
97,2022-06-02 16:40:00,1821.42000,1833.78000,1820.35000,1826.50000,4719
98,2022-06-02 16:35:00,1819.97000,1821.53000,1815.22000,1821.42000,3215


In [130]:
alphavantage_intradayETH["Open"] = pd.to_numeric(alphavantage_intradayETH["Open"])
alphavantage_intradayETH["High"] = pd.to_numeric(alphavantage_intradayETH["High"])
alphavantage_intradayETH["Low"] = pd.to_numeric(alphavantage_intradayETH["Low"])
alphavantage_intradayETH["Close"] = pd.to_numeric(alphavantage_intradayETH["Close"])
alphavantage_intradayETH["Volume"] = pd.to_numeric(alphavantage_intradayETH["Volume"])
alphavantage_intradayETH.dtypes

DateTime     object
Open        float64
High        float64
Low         float64
Close       float64
Volume        int64
dtype: object

In [131]:
alphavantage_intradayETH.describe()

,Open,High,Low,Close,Volume
count,100.0000,100.000000,100.000000,100.000000,100.00000
mean,1826.2364,1829.019900,1823.475000,1826.306900,2124.70000
std,9.8481,9.862958,9.997355,9.815419,1282.82135
min,1802.4600,1807.330000,1800.710000,1802.450000,270.00000
25%,1820.3100,1822.897500,1818.000000,1820.642500,1376.00000
50%,1827.0900,1828.605000,1824.605000,1827.085000,1922.50000
75%,1832.9100,1835.185000,1831.022500,1832.915000,2739.25000
max,1845.9900,1853.050000,1842.200000,1846.000000,10011.00000


In [132]:
countAnalisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].count()
countAnalisis_alphavantage_intradayETH

DateTime    100
Open        100
High        100
Low         100
Close       100
Volume      100
dtype: int64

In [133]:
meanAnalisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].mean()
meanAnalisis_alphavantage_intradayETH

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\2292396230.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  meanAnalisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].mean()


Open      1826.2364
High      1829.0199
Low       1823.4750
Close     1826.3069
Volume    2124.7000
dtype: float64

In [134]:
minAnalisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].min()
minAnalisis_alphavantage_intradayETH

DateTime    2022-06-02 16:30:00
Open                    1802.46
High                    1807.33
Low                     1800.71
Close                   1802.45
Volume                      270
dtype: object

In [135]:
maxAnalisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].max()
maxAnalisis_alphavantage_intradayETH

DateTime    2022-06-03 00:45:00
Open                    1845.99
High                    1853.05
Low                      1842.2
Close                    1846.0
Volume                    10011
dtype: object

In [136]:
q25Analisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.25)
q25Analisis_alphavantage_intradayETH

Open      1820.3100
High      1822.8975
Low       1818.0000
Close     1820.6425
Volume    1376.0000
Name: 0.25, dtype: float64

In [137]:
q50Analisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.5)
q50Analisis_alphavantage_intradayETH

Open      1827.090
High      1828.605
Low       1824.605
Close     1827.085
Volume    1922.500
Name: 0.5, dtype: float64

In [138]:
q75Analisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.75)
q75Analisis_alphavantage_intradayETH

Open      1832.9100
High      1835.1850
Low       1831.0225
Close     1832.9150
Volume    2739.2500
Name: 0.75, dtype: float64

In [139]:
q95nalisis_alphavantage_intradayETH = alphavantage_intradayETH[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.95)
q95nalisis_alphavantage_intradayETH

Open      1842.5765
High      1844.9930
Low       1839.6675
Close     1842.5860
Volume    3925.1000
Name: 0.95, dtype: float64

In [140]:
status, alphavantage_data = alphavantage.intraday('BTC')
if status:
    #print(alphavantage_data)
    alphavantage_intradayBTC = pd.DataFrame.from_dict(alphavantage_data['Time Series Crypto (5min)'])
alphavantage_intradayBTC = alphavantage_intradayBTC.T
alphavantage_intradayBTC = alphavantage_intradayBTC.reset_index(level=0)
alphavantage_intradayBTC.rename(columns = n_columns_rename, inplace = True)
alphavantage_intradayBTC

,DateTime,Open,High,Low,Close,Volume
0,2022-06-03 00:45:00,30391.13000,30409.12000,30371.06000,30397.86000,66
1,2022-06-03 00:40:00,30409.69000,30425.38000,30385.43000,30391.14000,80
2,2022-06-03 00:35:00,30465.68000,30480.95000,30395.30000,30409.70000,113
3,2022-06-03 00:30:00,30472.32000,30500.98000,30461.06000,30465.68000,148
4,2022-06-03 00:25:00,30478.98000,30484.44000,30408.96000,30472.32000,153
...,...,...,...,...,...,...
95,2022-06-02 16:50:00,30239.00000,30319.69000,30226.52000,30303.16000,171
96,2022-06-02 16:45:00,30304.81000,30332.20000,30227.73000,30238.99000,259
97,2022-06-02 16:40:00,30268.30000,30354.74000,30256.00000,30304.80000,305
98,2022-06-02 16:35:00,30273.43000,30273.44000,30221.70000,30268.30000,171


In [141]:
alphavantage_intradayBTC["Open"] = pd.to_numeric(alphavantage_intradayBTC["Open"])
alphavantage_intradayBTC["High"] = pd.to_numeric(alphavantage_intradayBTC["High"])
alphavantage_intradayBTC["Low"] = pd.to_numeric(alphavantage_intradayBTC["Low"])
alphavantage_intradayBTC["Close"] = pd.to_numeric(alphavantage_intradayBTC["Close"])
alphavantage_intradayBTC["Volume"] = pd.to_numeric(alphavantage_intradayBTC["Volume"])
alphavantage_intradayBTC.dtypes

DateTime     object
Open        float64
High        float64
Low         float64
Close       float64
Volume        int64
dtype: object

In [142]:
alphavantage_intradayBTC.describe()

,Open,High,Low,Close,Volume
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,30333.286100,30365.827100,30301.837200,30334.656600,156.780000
std,132.563513,136.658097,133.676909,132.512948,134.505272
min,30020.610000,30129.140000,29977.450000,30020.620000,27.000000
25%,30243.390000,30278.752500,30221.590000,30243.390000,89.750000
50%,30303.475000,30325.495000,30276.890000,30304.295000,139.500000
75%,30441.652500,30480.237500,30399.777500,30441.645000,171.250000
max,30642.780000,30689.000000,30591.900000,30642.780000,1265.000000


In [143]:
countAnalisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].count()
countAnalisis_alphavantage_intradayBTC

DateTime    100
Open        100
High        100
Low         100
Close       100
Volume      100
dtype: int64

In [144]:
meanAnalisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].mean()
meanAnalisis_alphavantage_intradayBTC

C:\Users\julia\AppData\Local\Temp\ipykernel_13564\630221912.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  meanAnalisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].mean()


Open      30333.2861
High      30365.8271
Low       30301.8372
Close     30334.6566
Volume      156.7800
dtype: float64

In [145]:
minAnalisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].min()
minAnalisis_alphavantage_intradayBTC

DateTime    2022-06-02 16:30:00
Open                   30020.61
High                   30129.14
Low                    29977.45
Close                  30020.62
Volume                       27
dtype: object

In [146]:
maxAnalisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].max()
maxAnalisis_alphavantage_intradayBTC

DateTime    2022-06-03 00:45:00
Open                   30642.78
High                    30689.0
Low                     30591.9
Close                  30642.78
Volume                     1265
dtype: object

In [147]:
q25Analisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.25)
q25Analisis_alphavantage_intradayBTC

Open      30243.3900
High      30278.7525
Low       30221.5900
Close     30243.3900
Volume       89.7500
Name: 0.25, dtype: float64

In [148]:
q50Analisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.5)
q50Analisis_alphavantage_intradayBTC

Open      30303.475
High      30325.495
Low       30276.890
Close     30304.295
Volume      139.500
Name: 0.5, dtype: float64

In [149]:
q75Analisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.75)
q75Analisis_alphavantage_intradayBTC

Open      30441.6525
High      30480.2375
Low       30399.7775
Close     30441.6450
Volume      171.2500
Name: 0.75, dtype: float64

In [150]:
q95nalisis_alphavantage_intradayBTC = alphavantage_intradayBTC[["DateTime", "Open", "High", "Low", "Close", "Volume"]].quantile(0.95)
q95nalisis_alphavantage_intradayBTC

Open      30558.5685
High      30604.4810
Low       30529.6385
Close     30558.5590
Volume      305.6000
Name: 0.95, dtype: float64

In [54]:
# Este no nos funciona :(
status, alphavantage_data = alphavantage.daily('BTC')
print(status)
if status:
    print(status)
    alphavantage_dailyBTC = pd.DataFrame.from_dict(alphavantage_data)
alphavantage_dailyBTC
#alphavantage_dailyBTC = alphavantage_dailyBTC.T
#n_columns_rename = {'1. open':'Open', '2. high':'High', '3. low':'Low', '4. close':'Close', '5. volume':'Volume',}
#alphavantage_dailyBTC

False


NameError: name 'alphavantage_dailyBTC' is not defined

In [152]:
status, alphavantage_data = alphavantage.exchange_rate('BTC')
if status:
    #print(alphavantage_data)
    alphavantage_exchange_rateBTC = pd.DataFrame.from_dict(alphavantage_data)
alphavantage_exchange_rateBTC
alphavantage_exchange_rateBTC = alphavantage_exchange_rateBTC.T
n_columns_rename2 = {'1. From_Currency Code':'FromCurrencyCode', '2. From_Currency Name':'FromCurrencyName', '3. To_Currency Code':'ToCurrencyCode', '4. To_Currency Name':'ToCurrencyName', '5. Exchange Rate':'ExchangeRate', '6. Last Refreshed':'LastRefreshed', '7. Time Zone':'TimeZone', '8. Bid Price':'BidPrice', '9. Ask Price':'AskPrice'}
alphavantage_exchange_rateBTC.rename(columns = n_columns_rename2, inplace = True)
alphavantage_exchange_rateBTC

,FromCurrencyCode,FromCurrencyName,ToCurrencyCode,ToCurrencyName,ExchangeRate,LastRefreshed,TimeZone,BidPrice,AskPrice
Realtime Currency Exchange Rate,BTC,Bitcoin,CNY,Chinese Yuan,202605.46016100,2022-06-03 00:54:51,UTC,202605.39355800,202605.46016100


In [157]:
alphavantage_exchange_rateBTC["ExchangeRate"] = pd.to_numeric(alphavantage_exchange_rateBTC["ExchangeRate"])
alphavantage_exchange_rateBTC["BidPrice"] = pd.to_numeric(alphavantage_exchange_rateBTC["BidPrice"])
alphavantage_exchange_rateBTC["AskPrice"] = pd.to_numeric(alphavantage_exchange_rateBTC["AskPrice"])
alphavantage_exchange_rateBTC = alphavantage_exchange_rateBTC.astype({"FromCurrencyCode": str,"FromCurrencyName": str,"ToCurrencyCode": str,"ToCurrencyName": str,"TimeZone": str})
alphavantage_exchange_rateBTC['LastRefreshed'] = pd.to_datetime(alphavantage_exchange_rateBTC['LastRefreshed'])
alphavantage_exchange_rateBTC.dtypes

FromCurrencyCode            object
FromCurrencyName            object
ToCurrencyCode              object
ToCurrencyName              object
ExchangeRate               float64
LastRefreshed       datetime64[ns]
TimeZone                    object
BidPrice                   float64
AskPrice                   float64
dtype: object

In [158]:
alphavantage_exchange_rateBTC.describe()

,ExchangeRate,BidPrice,AskPrice
count,1.000000,1.000000,1.000000
mean,202605.460161,202605.393558,202605.460161
std,NaN,NaN,NaN
min,202605.460161,202605.393558,202605.460161
25%,202605.460161,202605.393558,202605.460161
50%,202605.460161,202605.393558,202605.460161
75%,202605.460161,202605.393558,202605.460161
max,202605.460161,202605.393558,202605.460161


In [62]:
status, alphavantage_data = alphavantage.exchange_rate('ETH')
if status:
    #print(alphavantage_data)
    alphavantage_exchange_rateETH = pd.DataFrame.from_dict(alphavantage_data)
alphavantage_exchange_rateETH
alphavantage_exchange_rateETH = alphavantage_exchange_rateETH.T
n_columns_rename2 = {'1. From_Currency Code':'FromCurrencyCode', '2. From_Currency Name':'FromCurrencyName', '3. To_Currency Code':'ToCurrencyCode', '4. To_Currency Name':'ToCurrencyName', '5. Exchange Rate':'ExchangeRate', '6. Last Refreshed':'LastRefreshed', '7. Time Zone':'TimeZone', '8. Bid Price':'BidPrice', '9. Ask Price':'AskPrice'}
alphavantage_exchange_rateETH.rename(columns = n_columns_rename2, inplace = True)
alphavantage_exchange_rateETH

,FromCurrencyCode,FromCurrencyName,ToCurrencyCode,ToCurrencyName,ExchangeRate,LastRefreshed,TimeZone,BidPrice,AskPrice
Realtime Currency Exchange Rate,ETH,Ethereum,CNY,Chinese Yuan,12275.93194500,2022-06-02 23:25:46,UTC,12275.86534200,12275.93194500


In [159]:
alphavantage_exchange_rateETH["ExchangeRate"] = pd.to_numeric(alphavantage_exchange_rateETH["ExchangeRate"])
alphavantage_exchange_rateETH["BidPrice"] = pd.to_numeric(alphavantage_exchange_rateETH["BidPrice"])
alphavantage_exchange_rateETH["AskPrice"] = pd.to_numeric(alphavantage_exchange_rateETH["AskPrice"])
alphavantage_exchange_rateETH = alphavantage_exchange_rateETH.astype({"FromCurrencyCode": str,"FromCurrencyName": str,"ToCurrencyCode": str,"ToCurrencyName": str,"TimeZone": str})
alphavantage_exchange_rateETH['LastRefreshed'] = pd.to_datetime(alphavantage_exchange_rateETH['LastRefreshed'])
alphavantage_exchange_rateETH.dtypes

FromCurrencyCode            object
FromCurrencyName            object
ToCurrencyCode              object
ToCurrencyName              object
ExchangeRate               float64
LastRefreshed       datetime64[ns]
TimeZone                    object
BidPrice                   float64
AskPrice                   float64
dtype: object

In [160]:
alphavantage_exchange_rateETH.describe()

,ExchangeRate,BidPrice,AskPrice
count,1.000000,1.000000,1.000000
mean,12275.931945,12275.865342,12275.931945
std,NaN,NaN,NaN
min,12275.931945,12275.865342,12275.931945
25%,12275.931945,12275.865342,12275.931945
50%,12275.931945,12275.865342,12275.931945
75%,12275.931945,12275.865342,12275.931945
max,12275.931945,12275.865342,12275.931945
